In [2]:
import pandas as pd

df_idx = pd.read_csv('../data/processed/filtered_detailed_results.csv')
df_corpus = pd.read_csv('../data/raw/lyrics_pairs.csv')

In [5]:
print(df_idx)
print(df_corpus)

      annotator_id                    value  sim_rating    id1    id2  \
0          3747424  Apenas existe similitud           2  14251  76075   
1          3747424  Completamente diferente           1  29094  33088   
2          3747427           Poca similitud           3  37416  59899   
3          3747424           Poca similitud           3  31841   3071   
4          3747427  Apenas existe similitud           2  37741  35331   
...            ...                      ...         ...    ...    ...   
2023       3748376  Apenas existe similitud           2  48054  19354   
2024       3748378           Poca similitud           3  69790  38000   
2025       3748394  Similitud Sobresaliente           6  25639  16872   
2026       3747469           Poca similitud           3  33088  24450   
2027       3748343  Apenas existe similitud           2  14251  10052   

          pair_id  
0     14251_76075  
1     29094_33088  
2     37416_59899  
3      31841_3071  
4     37741_35331  
...

In [2]:
df = pd.DataFrame()


,id_1,artist_name_1,track_name_1,lyrics_1,id_2,artist_name_2,track_name_2,lyrics_2
0,5439,Eduardo Mateo,Dulce Brillo,Dulce brillo\nque aún estaba afuera\ncurtiendo...,29094,Maga,"Sí, Pero No Lo Soy","No exactamente yo, aunque si\nPero no lo soy\n..."
1,5439,Eduardo Mateo,Dulce Brillo,Dulce brillo\nque aún estaba afuera\ncurtiendo...,69790,Los Piojos,Ximenita,Mi novia Ximenita\nle gusta pelear\npatea de c...
2,5439,Eduardo Mateo,Dulce Brillo,Dulce brillo\nque aún estaba afuera\ncurtiendo...,66730,Arcana,Chant of the Awakening,Ohh de profundis\nOhh de procella\nAhh Canto\n...
3,5439,Eduardo Mateo,Dulce Brillo,Dulce brillo\nque aún estaba afuera\ncurtiendo...,28437,Arbolito,La Máquina,"Hasta la punta de tu luz,\nHasta tu grito más ..."
4,5439,Eduardo Mateo,Dulce Brillo,Dulce brillo\nque aún estaba afuera\ncurtiendo...,37485,Atahualpa Yupanqui,Camino del Indio - Remastered,Caminito del indio\nsendero coya sembrao de pi...
...,...,...,...,...,...,...,...,...
2770,48594,Reincidentes,Jornaleros Andaluces,Y en la plaza de mi pueblo\nDijo el jornalero ...,25047,Los Visitantes,Pi-Pa-Pu,PI-PA-PU\nPA PA PI PI PA PIUI\nPI PA PI\nPA PI...
2771,48594,Reincidentes,Jornaleros Andaluces,Y en la plaza de mi pueblo\nDijo el jornalero ...,36583,Roberto Calo,Romantica,"Romántica incurable, ¿te recuerdas?\nesa noche..."
2772,37194,Bulldog,Un Vinito Mas,Me voy a tomar\nEse vino que pensas dejar\nY n...,25047,Los Visitantes,Pi-Pa-Pu,PI-PA-PU\nPA PA PI PI PA PIUI\nPI PA PI\nPA PI...
2773,37194,Bulldog,Un Vinito Mas,Me voy a tomar\nEse vino que pensas dejar\nY n...,36583,Roberto Calo,Romantica,"Romántica incurable, ¿te recuerdas?\nesa noche..."
